# 1. Giới thiệu:
ResNet là 1 cấu trúc được giới thiệu bởi Kaiming He et al. vào năm 2015 trong bài báo: "Deep Residual Learning for Image Recognition" [(link)](https://arxiv.org/abs/1512.03385). Ý tưởng chính của paper là học phần dư(residual) thay vì học trực tiếp output của 1 layer, giúp việc huấn luyện các mạng rất sâu (trên 100 lớp) dễ dàng hơn. Dòng này học tốt trên ảnh 3 * 224 * 224, do phần giảm mạnh kích thước ở phần stem.

# 2. Vấn đề:
Khi độ sâu của mạng tăng lên, người ta kì vọng rằng mạng sẽ học được các biểu diễn phức tạp hơn, qua đó độ chính xác cũng cao hơn. Tuy nhiên trong thực tế, khi số tầng tăng lên quá nhiều, mạng lại học kém hơn cũng như khó học hơn dù có cùng dữ liệu. Ví dụ như cùng 1 cấu trúc, 1 mạng 20 tầng sẽ có val_acc 80%, trong khi mạng 56 tầng lại chỉ có 75%. Đây là $\textbf{Degradation problem}$ (suy giảm hiệu suất). Nguyên nhân chính của vấn đề này là do:

1. Vanishing gradient problem: Càng vào sâu, gradient backward trong backward propagation sẽ càng ngày càng tiêu biến và mức độ ảnh hưởng nhỏ đi, khiến các lớp đầu của mạng học rất chậm hoặc không học được, khiến mạng không thể tối ưu hiệu quả. 
Vấn đề này trở nên rõ rệt hơn với các activation layer sigmoid và tanh, vốn trả về các đạo hàm nhỏ hơn 1, khiến gradient gần như tiêu biến khi đến các lớp đầu.

2. Optimization difficulty: Mạng quá sâu sẽ dẫn đến việc có quá nhiều tham số để tối ưu, gây hao tổn tài nguyên, cũng như có càng nhiều local minima hoặc saddle point (đạo hàm bằng 0 nhưng không phải cực trị tại khu vực đó, và là thứ nghiêm trọng hơn local minima vì khó tối ưu hơn nhiều) khiến model khó tìm được global minima.

3. Khó bỏ qua các lớp không quan trọng: Trong trường hợp có 1 số lớp không giúp cải thiện kết quả, ta nên bỏ qua chúng bằng identity mapping (có nghĩa là output y như input: $H(x) = x$). Nhưng trong các thiết kế truyền thống không có cơ chế này, nên model vẫn sẽ cố học những ánh xạ không cần thiết gây lỗi.

Bằng chứng của ResNet paper nằm ở thí nghiệm ở ngay trang 1, khi cả training error và test error của model 56 lớp lại cao hơn cả 1 model chỉ có 20 lớp. 

# 3. Giải pháp đề xuất:
Paper đề xuất 1 lối tư duy mới: Thay vì học hết toàn bộ mọi thứ liên quan đến sự thay đổi của output, hãy chỉ học phần khác biệt giữa input và output sau khi qua các lớp. Ở đây, thay vì học hết ánh xạ $H(x)$, ta lại đi học phần hiệu giữa nó và đầu vào:
$$F(x)=H(x)-x$$
Hay nói cách khác, $H(x)=F(x)+x$. 

<img src="image0.png"> 

Thông tin gốc sẽ được truyền thẳng đến output và skip qua tất cả các layer ở giữa. Đây gọi là skip connection. Cơ chế này giúp ổn định luồng gradient (không dễ suy biến vì luôn đảm bảo gradient tối thiểu thêm 1 $\frac{\partial{L}}{\partial{x}}=\frac{\partial{L}}{\partial{y}}(\frac{\partial{F(x)}}{\partial{x}}+1)$), qua đó giảm bớt hiện tượng vanishing gradient nếu như $\frac{\partial{F(x)}}{\partial{x}}$ âm quá lớn và giúp mạng dễ tối ưu hơn qua việc bỏ các layer đi nếu $F(x)$ quá nhỏ. 
Note: Không nhất thiết là phải luôn +x . Ý tưởng về việc thay đổi hệ số của x cũng như của $F(x)$ đã được thực hiện qua các paper sau: 
- Weighted residual networks (2016) (Paper: [(link)](https://arxiv.org/pdf/1605.08831)):
Cải tiến thay vì $H(x)=F(x)+x$,  thêm 1 phần trọng số vào cuối phép biến đổi x rồi đưa vào activation function.

<img src="image1.png">

- Fixup Initialization (2019) (Paper: [(link)](https://arxiv.org/pdf/1605.08831)):
Cải tiến thay vì $H(x)=F(x)+x$,  thêm 1 phần trọng số vào phần scalar x thành $H(x)=F(x)+\alpha x$, thêm scale và bias cho các layer và bỏ BatchNorm.
- Dynamical isometry for residual networks(2022) (Paper: [https://arxiv.org/pdf/2210.02411]):
Biến đổi sao cho tất cả các giá trị riêng của ma trận Jacobian ở đầu vào và ra đều gần 1, giúp ổn định hóa luồng gradient. Bằng cách không random init các parameter mà chọn kiểu init sao cho các thành phần trong block thỏa chuẩn tắc: $\alpha^2 + \beta^2 = 1$ cho $H(x)=\alpha F(x)+\beta x$ và không cần BatchNorm.

Nói hơi dài dòng 1 tí rồi, ta có thể coi skip connection như một kiểu ablation study, khi model tự động đánh giá mức độ quan trọng của các block. Nếu 1 block không đóng góp gì nhiều cho việc giảm lỗi ($F(x)$ quá nhỏ hoặc âm), thì $F(x)$ sẽ được điều chỉnh về 0, đồng nghĩa với việc "tắt" đi lớp đó một cách tự động mà không cần con người can thiệp.

# 4. Các thành phần cơ bản trong ResNet:
ResNet được xây dựng từ các khối chính sau:
- Residual Block: Là Basic block của ResNet. Cồm có 2 hoặc 3 lớp convolution liên tiếp, mỗi lớp thường gồm:
    - Conv layer: thực hiện trích xuất feature
    - Batch Normalization (BN): Chuẩn hóa dữ liệu, ổn định phân phối dữ liệu trong quá trình huấn luyện
    - Activation function (ReLU): Hàm phi tuyến phổ biến trong ResNet, giúp tăng tính phi tuyến của mạng
Kết quả đầu ra của khối sẽ được cộng với input của không qua skip connection:
$$y=F(x)+x$$
Có 2 kiểu block phổ biến:
    - Basic block: 2 conv 3x3, dùng trong ResNet18, ResNet34
    - Bottleneck block: conv 1x1 giảm chiều -> conv 3x3 -> conv 1x1 tăng chiều trong ResNet50 trở lên để giảm tham số và tăng hiệu suất tính toán.
- Skip connection: Là đường truyền input qua block để cộng trực tiếp với output từ block. Có 2 dạng skip chính:
    - Identity shortcut: Khi input và output đã qua block có cùng kích thước, skip connection truyền x giống hệt input.
    - Projection shortcut: Khi kích thước output thay đổi, dùng 1 convolution 1x1 để chiếu(project) input về kích thước phù hợp với output trước khi cộng.
- Pooling và Fully connected layer (FC):
    - Pooling: giảm kích thước ảnh đầu vào, giảm chi phí tính toán.
    - FC layer: Ở cuối mạng, chuyển các feature trích xuất được thành classification.

Một kiến trúc ResNet điển hình sẽ có :
1 conv -> BN -> ReLU -> MaxPool -> n residual block -> Global average pool/Flatten -> FC


# 5. Các phiên bản ResNet cơ bản:
Con số trong ResNet chỉ tổng số lớp có trọng số (như conv layer hay FC layer) được sử dụng trong network.
- ResNet 18, 34: dùng BasicBlock
- ResNet 50, 101, 152: dùng BottleNeck

# 6. Các biến thể của ResNet:
- ResNeXt: nhóm convolution + cardinality 
- Wide-ResNet: tăng chiều rộng thay vì độ sâu
- Res2Net: chia nhỏ tầng conv thành nhiều nhánh nhỏ
- ResUNet: dùng trong segmentation, kết hợp U-Net + ResNet
- ConvNeXt: giống bản "ResNet++", đã có các design pattern của ViT như LayerNorm, GELU, 7x7 depthwise conv, preactivation, không pooling mà dùng down sampling. Nó là bản ResNet mạnh nhất, cũng như là bản CNN mạnh nhất để đấu ViT, dù vẫn thua :)). 

# 7. Kết luận:
ResNet là kiến trúc mạng nơ-ron sâu sử dụng cơ chế skip connection giúp giải quyết vấn đề 
vanishing gradient và suy giảm hiệu suất khi mạng tăng độ sâu. Nhờ đó, ResNet cho phép xây
dựng các mô hình rất sâu mà vẫn dễ dàng huấn luyện và đạt hiệu quả cao. Đây là một bước đột 
phá quan trọng, làm nền tảng cho nhiều kiến trúc mạng hiện đại và ứng dụng trong thị giác máy tính.